In [69]:
class Field:
    def __init__(self, ftype):
        self.ftype = ftype

    def __set_name__(self, owner, name):
        self.name = name

    # note if a field is invoked from User class, e.g., User.name
    # instance argument here will be None
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance._data.get(self.name)

    def __set__(self, instance, value):
        if not isinstance(value, self.ftype):
            raise TypeError(
                f"{self.name} must be {self.ftype.__name__}"
            )
        instance._data[self.name] = value
        instance._dirty.add(self.name)

    # query operators
    def __eq__(self, value):
        return CmpExpr(self.name, "==", value)

    def __ge__(self, value):
        return CmpExpr(self.name, ">=", value)

    def __le__(self, value):
        return CmpExpr(self.name, "<=", value)

    ########### fill in the code to support > and < (10 points) ############

    def __gt__(self, value):
        return CmpExpr(self.name, ">", value)
    
    def __lt__(self, value):
        return CmpExpr(self.name, "<", value)


In [70]:
class Expr:
    def __bool__(self):
        raise TypeError("Expression cannot be used as boolean")

class CmpExpr(Expr):
    def __init__(self, field, op, value):
        self.field = field
        self.op = op
        self.value = value


In [71]:
import requests
import json
from urllib.parse import urlencode

### replace the following URL with your DB url
### note it does not end with /, that is, no slash after .com
FIREBASE_URL = "https://dsci551-hw1-7868f-default-rtdb.firebaseio.com"

class Document:
    endpoint = None # to be supplied by subclass

    def __init__(self, id, **kwargs):
        self.id = id # id is required
        self._data = {} # attribute-values stored locally
        self._dirty = set() # attributes to be updated on the DB server

        for k, v in kwargs.items():
            setattr(self, k, v) # self.k = v
            # note this will trigger descriptor protocol for field attribute
            # calling its __set__ method

    def save(self):
        """Persist only modified fields."""
        """That is, all fields marked as dirty."""
        """Remember to clear dirty flags after updating the data with server."""
        if not self._dirty:
            return

        ########### fill in the code (30 points) ############
        update_data = {field: self._data[field] for field in self._dirty}
        url = f"{FIREBASE_URL}/{self.endpoint}/{self.id}.json"
        response = requests.patch(url, json=update_data)
        response.raise_for_status()
        self._dirty.clear()


    @classmethod
    def fetch(cls, id):
        """Fetch a document with the given id from server."""
        """Return an object of this class if found."""
        """Return None if not found."""

        ########### fill in the code (25 points) ############
        url = f"{FIREBASE_URL}/{cls.endpoint}/{id}.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data is None:   
            return None
        instance = cls(id, **data)
        instance._dirty.clear() 
        return instance
            


    @classmethod
    def query(cls, expr):
        """Return a list of documents matching the query."""
        """It does not support orderBy="$key" and orderBy="$value"."""
        """It does not support limit."""

        if not isinstance(expr, CmpExpr):
            raise TypeError("Only one comparison supported")

        base_url = f"{FIREBASE_URL}/{cls.endpoint}.json"

        params = {"orderBy": json.dumps(expr.field)}

        if expr.op == "==":
            params["equalTo"] = json.dumps(expr.value)
        elif expr.op == ">=":
            params["startAt"] = json.dumps(expr.value)
        elif expr.op == "<=":
            params["endAt"] = json.dumps(expr.value)
        elif expr.op == ">":
            params["startAt"] = json.dumps(expr.value)
        elif expr.op == "<":
            params["endAt"] = json.dumps(expr.value)

        ########### fill in the code to support > and < (10 points) ############

        else:
            raise NotImplementedError(expr.op)

        url = f"{base_url}?{urlencode(params)}"

        ########### fill in the code (25 points) ############
        # retrieve from server and construct list of document objects
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data is None:
            return []
        result = []
        for doc_id, doc_data in data.items():
            instance = cls(doc_id, **doc_data)
            instance._dirty.clear()
            if expr.op == ">":
                if instance._data.get(expr.field) == expr.value:
                    continue
            elif expr.op == "<":
                if instance._data.get(expr.field) == expr.value:
                    continue
            result.append(instance)
        return result

    

class User(Document):
    endpoint = "users"

    name = Field(str)
    age = Field(int)
    gender = Field(str)

    def __repr__(self):
        return f"User(id={self.id}, data={self._data})"


In [72]:
u = User(1)
u.gender = "Male"
print(u) # note this only has gender info of User 1

# update User 1 info (kept locally) with the server
u.save()

User(id=1, data={'gender': 'Male'})


In [73]:
u = User.fetch(1) # fetch current value of User 1 from server
print(u)
# modify them locally
u.name = "Bob"
u.age = 30
u.save()  # push updates to the server
print(u)

User(id=1, data={'age': 25, 'gender': 'Male', 'name': 'Alice'})
User(id=1, data={'age': 30, 'gender': 'Male', 'name': 'Bob'})


In [74]:
u = User(2, age=20) # specify attribute-value in constructor
print(u)
u.save()
User.fetch(2)

User(id=2, data={'age': 20})


User(id=2, data={'age': 20, 'gender': 'Male', 'name': 'Charlie'})

In [75]:
u.age += 1 # update age locally
u.save()
User.fetch(2)

User(id=2, data={'age': 21, 'gender': 'Male', 'name': 'Charlie'})

In [76]:
users = User.query(User.age > 30)
users

[User(id=6, data={'age': 31, 'gender': 'Female', 'name': 'Grace'}),
 User(id=3, data={'age': 35, 'gender': 'Male', 'name': 'David'})]

In [77]:
users = User.query(User.name <= "Mary")
users

[User(id=4, data={'age': 22, 'gender': 'Female', 'name': 'Eva'}),
 User(id=5, data={'age': 28, 'gender': 'Male', 'name': 'Frank'}),
 User(id=6, data={'age': 31, 'gender': 'Female', 'name': 'Grace'}),
 User(id=1, data={'age': 30, 'gender': 'Male', 'name': 'Bob'}),
 User(id=2, data={'age': 21, 'gender': 'Male', 'name': 'Charlie'}),
 User(id=3, data={'age': 35, 'gender': 'Male', 'name': 'David'})]

In [78]:
# Task 1 Test

expr1 = User.age > 25
print(f"Field: age, Operator: {expr1.op}, Value: {expr1.value}")

expr2 = User.name < "John"
print(f"Field: name, Operator: {expr2.op}, Value: {expr2.value}")

expr3 = User.age >= 30
print(f"Field: age, Operator: {expr3.op}, Value: {expr3.value}")

Field: age, Operator: >, Value: 25
Field: name, Operator: <, Value: John
Field: age, Operator: >=, Value: 30


In [79]:
# Task 2 Test

print("Create a new user with only gender")
u1 = User(1)
u1.gender = "Female"
print(f" Before save: {u1}")
print(f" Dirty fields before save: {u1._dirty}")
u1.save()
print(f" After save: {u1}")
print(f" Dirty fields after save: {u1._dirty}")
print()

print("Update user with name and age")
u1 = User.fetch(1)
u1.name = "Alice"
u1.age = 25
print(f" Before save: {u1}")
print(f" Dirty fields before save: {u1._dirty}")
u1.save()
print(f" After save: {u1}")
print(f" Dirty fields after save: {u1._dirty}")
print()

print("Call save without any modification")
u1 = User.fetch(1)
print(f" Before save: {u1}")
print(f" Dirty fields before save: {u1._dirty}")
u1.save()
print(f" After save: {u1}")
print(f" Dirty fields after save: {u1._dirty}")

Create a new user with only gender
 Before save: User(id=1, data={'gender': 'Female'})
 Dirty fields before save: {'gender'}
 After save: User(id=1, data={'gender': 'Female'})
 Dirty fields after save: set()

Update user with name and age
 Before save: User(id=1, data={'age': 25, 'gender': 'Female', 'name': 'Alice'})
 Dirty fields before save: {'name', 'age'}
 After save: User(id=1, data={'age': 25, 'gender': 'Female', 'name': 'Alice'})
 Dirty fields after save: set()

Call save without any modification
 Before save: User(id=1, data={'age': 25, 'gender': 'Female', 'name': 'Alice'})
 Dirty fields before save: set()
 After save: User(id=1, data={'age': 25, 'gender': 'Female', 'name': 'Alice'})
 Dirty fields after save: set()


In [80]:
# Task 3 Test

print("Fetch existing user")
u1 = User.fetch(1)
print(f" Fetched User: {u1}")
print(f" Dirty fields after fetch: {u1._dirty}")
print()

print("Create User 2, then fetch")
u2 = User(2, name="Charlie", age=28, gender="Male")
u2.save()
print(f"  Created User: {u2}")
u2_fetched = User.fetch(2)
print(f" Fetched User: {u2_fetched}")
print(f" Dirty fields after fetch: {u2_fetched._dirty}")
print()

print("Fetch non-existing user")
u_nonexist = User.fetch(9999)
print(f" Fetched non-existing User: {u_nonexist}")
print()

print("Modify after fetch")
u2 = User.fetch(2)
print(f" Before modification: {u2}")
u2.age = 29
print(f" After modification: {u2}")
print(f" Dirty fields after modification: {u2._dirty}")
u2.save()
print(f" After save: {u2}")
print(f" Dirty fields after save: {u2._dirty}")

Fetch existing user
 Fetched User: User(id=1, data={'age': 25, 'gender': 'Female', 'name': 'Alice'})
 Dirty fields after fetch: set()

Create User 2, then fetch
  Created User: User(id=2, data={'name': 'Charlie', 'age': 28, 'gender': 'Male'})
 Fetched User: User(id=2, data={'age': 28, 'gender': 'Male', 'name': 'Charlie'})
 Dirty fields after fetch: set()

Fetch non-existing user
 Fetched non-existing User: None

Modify after fetch
 Before modification: User(id=2, data={'age': 28, 'gender': 'Male', 'name': 'Charlie'})
 After modification: User(id=2, data={'age': 29, 'gender': 'Male', 'name': 'Charlie'})
 Dirty fields after modification: {'age'}
 After save: User(id=2, data={'age': 29, 'gender': 'Male', 'name': 'Charlie'})
 Dirty fields after save: set()


In [81]:
# Test Data

users = [
    (3, "David", 35, "Male"),
    (4, "Eva", 22, "Female"),
    (5, "Frank", 28, "Male"),
    (6, "Grace", 31, "Female"),
]

for id, name, age, gender in users:
    u = User(id, name=name, age=age, gender=gender)
    u.save()
    

In [82]:
# Task 4 Test

print("Query users with age > 25")
users = User.query(User.age > 25)
print(f" Found {len(users)} users:")
for u in users:
    print(f" {u}")
print()

print("Query users with age >= 30")
users = User.query(User.age >= 30)
print(f" Found {len(users)} users:")
for u in users:
    print(f" {u}")
print()

print("Query users with age <= 28")
users = User.query(User.age <= 28)
print(f" Found {len(users)} users:")
for u in users:
    print(f" {u}")
print()

print("Query users with age == 31")
users = User.query(User.age == 31)
print(f" Found {len(users)} users:")
for u in users:
    print(f" {u}")
print()

print("Query users with name < 'Eva'")
users = User.query(User.name < "Eva")
print(f" Found {len(users)} users:")
for u in users:
    print(f" {u}")

Query users with age > 25
 Found 4 users:
 User(id=5, data={'age': 28, 'gender': 'Male', 'name': 'Frank'})
 User(id=6, data={'age': 31, 'gender': 'Female', 'name': 'Grace'})
 User(id=2, data={'age': 29, 'gender': 'Male', 'name': 'Charlie'})
 User(id=3, data={'age': 35, 'gender': 'Male', 'name': 'David'})

Query users with age >= 30
 Found 2 users:
 User(id=6, data={'age': 31, 'gender': 'Female', 'name': 'Grace'})
 User(id=3, data={'age': 35, 'gender': 'Male', 'name': 'David'})

Query users with age <= 28
 Found 3 users:
 User(id=4, data={'age': 22, 'gender': 'Female', 'name': 'Eva'})
 User(id=1, data={'age': 25, 'gender': 'Female', 'name': 'Alice'})
 User(id=5, data={'age': 28, 'gender': 'Male', 'name': 'Frank'})

Query users with age == 31
 Found 1 users:
 User(id=6, data={'age': 31, 'gender': 'Female', 'name': 'Grace'})

Query users with name < 'Eva'
 Found 3 users:
 User(id=1, data={'age': 25, 'gender': 'Female', 'name': 'Alice'})
 User(id=2, data={'age': 29, 'gender': 'Male', 'name